In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/chatbot deva3/nexyug_chatbot_dataset_30002.xlsx')

In [29]:
df.isnull().sum()

,0
User Input,0
Bot Response,0
Intent,0


In [30]:
df["User Input"]=df["User Input"].str.lower()
df["Bot Response"]=df["Bot Response"].str.lower()
df["Intent"]=df["Intent"].str.lower()
df.head(2)

,User Input,Bot Response,Intent
0,goodbye #1,goodbye! stay safe.,goodbye
1,what is your mission? #2,our mission is to innovate and deliver top ai ...,company_info


In [31]:
pattern_to_remove = r'#\d+'
df["User Input"]=df["User Input"].str.replace(pattern_to_remove, '', regex=True)
df["User Input"]=df["User Input"].str.replace(r'[^\w\s]', '', regex=True)
df.head(2)

,User Input,Bot Response,Intent
0,goodbye,goodbye! stay safe.,goodbye
1,what is your mission,our mission is to innovate and deliver top ai ...,company_info


In [32]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [33]:
df['User Input'] = df['User Input'].fillna('').astype(str)
df['tokenized_user_input'] = df['User Input'].apply(word_tokenize)

In [34]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords_from_list(tokens):
    return [word for word in tokens if word not in stop_words]

df['filtered_user_input'] = df['tokenized_user_input'].apply(remove_stopwords_from_list)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,User Input,Bot Response,Intent,tokenized_user_input,filtered_user_input
0,goodbye,goodbye! stay safe.,goodbye,[goodbye],[goodbye]
1,what is your mission,our mission is to innovate and deliver top ai ...,company_info,"[what, is, your, mission]",[mission]
2,thanks a lot,anytime!,thank_you,"[thanks, a, lot]","[thanks, lot]"
3,how many employees do you have,we have over 200 employees globally.,company_info,"[how, many, employees, do, you, have]","[many, employees]"
4,bye,goodbye! have a great day!,goodbye,[bye],[bye]


In [35]:
dataset = []

for _, row in df.iterrows():
    user_input = " ".join(row['filtered_user_input'])
    response = row['Bot Response']
    dataset.append({
        "input": user_input,
        "output": response
    })


In [36]:
import json
with open("chatbot_data.json", "w") as f:
    json.dump(dataset, f, indent=2)

In [37]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("gpt2")

In [38]:
from datasets import Dataset
from transformers import AutoTokenizer

df = pd.read_json("/content/chatbot_data.json")
dataset = Dataset.from_pandas(df)

def tokenize_function(example):
    prompt = f"User: {example['input']}\nBot:"
    full_text = prompt + " " + example["output"]

    tokens = tokenizer(
        full_text,
        padding="max_length",
        truncation=True,
        max_length=128
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

In [39]:
tokenized = dataset.map(tokenize_function, remove_columns=dataset.column_names)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [43]:
import torch
from transformers import (
    GPT2TokenizerFast,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)

# 1. Check device
use_cuda = torch.cuda.is_available()
print(f"CUDA available: {use_cuda}")

# 2. Load model & tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# 3. Prepare data (assuming you already have `tokenized` as a Dataset)
#    e.g. tokenized = my_dataset.map(lambda ex: tokenizer(ex["text"], truncation=True, padding="max_length"), ...)

# 4. Create a padding collator to replace the deprecated `tokenizer` arg
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 5. Define training arguments, toggling fp16 only if GPU is present
training_args = TrainingArguments(
    output_dir="./gpt2-chatbot",
    per_device_train_batch_size=4,
    num_train_epochs=4,
    save_strategy="epoch",
    logging_dir="./logs",
    fp16=use_cuda,              # ← enable half-precision only on CUDA GPUs
    push_to_hub=False,          # optional: disable if you aren’t pushing to HF Hub
)

# 6. Instantiate the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    eval_dataset=tokenized,     # replace with your eval split
    tokenizer=tokenizer,        # still needed for generation, etc.
    data_collator=data_collator # fixes the FutureWarning
)

# 7. Launch training
trainer.train()


CUDA available: False


<ipython-input-43-85cbd5ad02ec>:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: fp16 mixed precision requires a GPU (not 'xla').

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
def generate_response(user_input):
    prompt = f"User: {user_input}\nBot:"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_length=128,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("Bot:")[-1].strip()

In [ ]:
user_input= input("Query: ")
generate_response(user_input)